In [ ]:
%matplotlib inline
import pickle
from pathlib import Path

import astropy.units as u
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from astropy.table import Table
from astropy.time import Time
from gammapy.estimators import FluxPoints
from gammapy.maps import TimeMapAxis
from gammapy.modeling.models import create_crab_spectral_model, SkyModel
from scipy.stats import chi2

TimeMapAxis.time_format = "mjd"

In [ ]:
sns.set_color_codes("colorblind")
sns.set_palette("colorblind")

MPL_LINEWIDTH = 1.1

mpl_rc = {
    "figure.autolayout": False,
    "font.size": 20,
    "figure.dpi": 300,
    "lines.linewidth": 1.5,
    "axes.grid": False,
    "axes.linewidth": MPL_LINEWIDTH,
    "xtick.major.size": 6,
    "xtick.major.width": MPL_LINEWIDTH,
    "xtick.minor.size": 5,
    "xtick.minor.width": MPL_LINEWIDTH,
    "xtick.minor.visible": False,
    "ytick.major.size": 6,
    "ytick.major.width": MPL_LINEWIDTH,
    "ytick.minor.size": 5,
    "ytick.minor.width": MPL_LINEWIDTH,
    "ytick.minor.visible": False,
}
plt.style.use(mpl_rc)

In [ ]:
def set_axes(axis):
    axis.set_xlabel("MJD")
    axis.set_ylabel(r"Flux$_{E > 100 \rm{~GeV}}$ (cm$^{-2}$ s$^{-1}$)")
    axis.set_yscale("linear")
    axis.set_ylim(0, 7e-10)


def mjd_to_iso(times):
    return Time(times, format="mjd", scale="utc").iso


def set_twin_time_axis(axis):
    ax2 = axis.twiny()

    years = mdates.YearLocator()  # every year
    months = mdates.MonthLocator(interval=2)  # every month

    # format the ticks
    ax2.xaxis.set_major_locator(years)
    ax2.xaxis.set_minor_locator(months)

    # Manually adjust
    datemin = np.datetime64(mjd_to_iso(59150), "W")
    datemax = np.datetime64(mjd_to_iso(59670), "W")

    ax2.set_xlim(datemin, datemax)

    ax2.format_xdata = mdates.DateFormatter("%b")

    mjdlims = Time([datemin, datemax]).mjd
    axis.set_xlim(mjdlims[0], mjdlims[1])
    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    ax2.xaxis.set_minor_formatter(mdates.DateFormatter("%b"))

    ax2.tick_params(axis="x", which="major", pad=13)
    ax2.tick_params(axis="x", which="minor", pad=10)


def weighted_average(table, sys_error=0.0):
    val = table["flux"]
    uncertainty = np.sqrt((sys_error * table["flux"]) ** 2 + table["flux_err"] ** 2)
    return (val / uncertainty**2).sum() / (1 / uncertainty**2).sum(), np.sqrt(
        1 / np.sum(1 / uncertainty**2)
    )


def calculate_chi2_pvalue(table, sys_error=0.0):
    uncertainty = np.sqrt((sys_error * table["flux"]) ** 2 + table["flux_err"] ** 2)
    flux = table["flux"]
    mean_flux = (flux / uncertainty**2).sum() / (1 / uncertainty**2).sum()
    mean_flux_err = np.sqrt(1 / np.sum(1 / uncertainty**2))
    print(f"Weighted mean flux: {mean_flux:.3e} +/- {mean_flux_err:.3e} cm-2 s-1")

    chi2_value = np.sum((table["flux"] - mean_flux) ** 2 / uncertainty**2)
    ndf = len(table["flux"]) - 1
    pvalue = chi2.sf(x=chi2_value, df=ndf)
    print(f"Chi2: {chi2_value:.1f}, ndf: {ndf}, P-value: {pvalue:.2e}")
    return chi2_value, ndf, pvalue


def plot_total_error_bars(sys_uncertainty: float, lightcurve):
    time_min = np.hstack(lightcurve["time_min"])
    time_max = np.hstack(lightcurve["time_max"])
    time_mjd = 0.5 * (time_min + time_max)
    flux = np.hstack(lightcurve["flux"])
    statistical_err = np.hstack(lightcurve["flux_err"])

    sys_err = sys_uncertainty * flux  # sys_uncertainty is the percentage of flux values
    total_error = np.sqrt(statistical_err**2 + sys_err**2)  # added in quadrature
    ax.errorbar(time_mjd, flux, yerr=total_error, fmt=".", color="gray", alpha=0.5)


def plot_crab_average_flux_band(lightcurve, axis):
    mean_flux, mean_flux_err = weighted_average(lightcurve)
    axis.axhline(
        mean_flux,
        c="b",
        ls="-.",
        label="Best fit value to a constant flux (LST-1)",
    )
    # Plot uncertainty band
    axis.axhspan(
        mean_flux - mean_flux_err, mean_flux + mean_flux_err, alpha=0.2, color="b"
    )


def plot_magic_crab_reference(axis):
    """
    Calculate & plot Crab MAGIC reference flux from
    https://doi.org/10.1016/j.jheap.2015.01.002
    """
    crab = create_crab_spectral_model("magic_lp")

    crab.amplitude.error = 0.03e-11 * u.Unit("cm-2 s-1 TeV-1")
    crab.alpha.error = 0.01
    crab.beta.error = 0.01 / np.log(10)

    e_lc_min = 100 * u.GeV
    e_lc_max = 30 * u.TeV

    magic_crab_flux, magic_crab_flux_error = crab.integral_error(e_lc_min, e_lc_max)

    axis.axhline(
        magic_crab_flux.to_value("cm-2 s-1"),
        c="tab:orange",
        ls="--",
        label="MAGIC (Aleksić et al. 2015)",
    )
    axis.axhspan(
        (magic_crab_flux - magic_crab_flux_error).to_value("cm-2 s-1"),
        (magic_crab_flux + magic_crab_flux_error).to_value("cm-2 s-1"),
        alpha=0.2,
        color="tab:orange",
    )

In [ ]:
data_dir = Path("./data/src_indep")

# Create directory for output plots
src_indep_plots = Path("src_indep_plots")
src_indep_plots.mkdir(exist_ok=True, parents=True)

# Open best-fit model file
model_file = data_dir / "SED_model_CrabNebula_only_LST1.dat"
with open(model_file, "rb") as f:
    model_dict = pickle.load(f)
model = SkyModel.from_dict(model_dict)

# Open light curve file and create corresponding FluxPoint object
lightcurve_table = Table.read(data_dir / "lightcurve_src_indep_crab_lst1.ecsv")
lc = FluxPoints.from_table(lightcurve_table, format="lightcurve", reference_model=model)

In [ ]:
# Get Crab average flux, chi2 and p-value assuming no systematic uncertainty
calculate_chi2_pvalue(lightcurve_table, sys_error=0.0)

In [ ]:
# Get Crab average flux, chi2 and p-value assuming 5.8% systematic uncertainty
# in the nightly flux values
calculate_chi2_pvalue(lightcurve_table, sys_error=0.058)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Plot light curve data points
lc.plot(
    ax=ax,
    sed_type="flux",
    marker="o",
    color="black",
    zorder=10,
    label="LST-1 (src-independent)",
)

# Plot best fit of the light curve data points to a constant flux
plot_crab_average_flux_band(lightcurve_table, ax)

# Plot Crab MAGIC reference
plot_magic_crab_reference(ax)

# Plot total error bars (statistical and systematic added in quadrature)
# considering 5.8% systematic uncertainty nightwise. This systematic
# uncertainty in the nightly fluxes is needed to obtain a P-value of
# around 0.5 in the fit of the light curve to a constant value
plot_total_error_bars(0.058, lightcurve_table)

# Reorder labels in the legend
handles, labels = ax.get_legend_handles_labels()
order = [2, 0, 1]
ax.legend(
    [handles[idx] for idx in order],
    [labels[idx] for idx in order],
    loc="lower left",
    fontsize=20,
)

set_twin_time_axis(ax)
set_axes(ax)

plt.savefig(src_indep_plots / "LC_crab_src-indep.pdf", bbox_inches="tight")
plt.savefig(src_indep_plots / "LC_crab_src-indep.png", bbox_inches="tight")